# Membership Questions
- How many members do we have of each type?
- How many members are in good/bad standing?
- How many members do we have paying what amounts?
- How many people are authorized for each thing?
- What is quorum?
- Who is eligible to run for the board?
- How many people voted in the last vote?

# Voting
- People should be able to automatically submit votes without having to wait for board reviewal
- We could then have people enter a proposed vote date when they submit it, and automatically email all voting membership with the link to vote on it
- Automatically email all members with results of vote

# Needs to Switch
- Domains to be switched on PS1 computers
- All the existing AD stuff needs to be migrated
- Authorizations to carry over form WildApricot to new AD

In [1]:
import pandas as pd

In [2]:
from wild_apricot import WildApricotAPI

In [3]:
api = WildApricotAPI()

In [4]:
auth_types = api.authorization_types()
cdf = api.contact_info(sensitive=True)  # members
adf = api.authorizations()  # authorizations
pdf = api.payment_info()  # payments

0 / 478
40 / 478
80 / 478
120 / 478
160 / 478
200 / 478
240 / 478
280 / 478
320 / 478
360 / 478
400 / 478
440 / 478


In [5]:
cdf.head()

,Authorizations,Email,FirstName,LastName,MembershipLevel,MembeshipEnabled,Status,TermsOfUseAccepted
ContactId,,,,,,,,
46280627,[Epilog Laser],*****@****.***,*****,*****,Area Host,True,Active,False
46994719,[],*****@****.***,*****,*****,Starving Hacker,False,Lapsed,False
46821349,[],*****@****.***,*****,*****,Starving Hacker,True,Active,False
46986614,"[Epilog Laser, ShopBot]",*****@****.***,*****,*****,Full Membership - PayPal,True,Active,True
47902655,[],*****@****.***,*****,*****,Starving Hacker,True,Active,False


In [6]:
pdf.head()

,AllocatedValue,Comment,ContactId,PaymentDate,PaymentId,TenderId,TenderName,Type,Value
1127,70.0,,46697076,2019-01-01,46518575,1159527.0,Stripe Recurring Payment,InvoicePayment,70.0
1376,40.0,,46717474,2019-01-01,46518574,1159527.0,Stripe Recurring Payment,InvoicePayment,40.0
547,40.0,,47564682,2019-01-01,46518558,1159527.0,Stripe Recurring Payment,InvoicePayment,40.0
392,40.0,,47575028,2019-01-01,46518578,1159527.0,Stripe Recurring Payment,InvoicePayment,40.0
467,40.0,,48410889,2019-01-01,46559246,1159527.0,Stripe Recurring Payment,InvoicePayment,40.0


# How many members do we have of each type?

In [7]:
cdf.groupby('MembershipLevel')['Email'].count()

MembershipLevel
Area Host                           12
Board Member                         7
Full Membership                     78
Full Membership - Grandfathered      2
Full Membership - PayPal            10
Full Volunteer Authorizer            5
Grant Membership (Full)              1
Starving Hacker                    316
Starving Hacker - PayPal            37
Volunteer Authorizer                 5
Name: Email, dtype: int64

In [10]:
# Filtered by MembershipEnabled
cdf[cdf["MembeshipEnabled"].fillna(False)].groupby('MembershipLevel')['Email'].count()

MembershipLevel
Area Host                           12
Board Member                         7
Full Membership                     66
Full Membership - Grandfathered      2
Full Membership - PayPal             7
Full Volunteer Authorizer            5
Grant Membership (Full)              1
Starving Hacker                    237
Starving Hacker - PayPal            27
Volunteer Authorizer                 5
Name: Email, dtype: int64

In [9]:
# Members with "Membership Disabled"
(cdf.groupby('MembershipLevel').count() -
cdf[cdf["MembeshipEnabled"]].groupby('MembershipLevel').count())['Email']

MembershipLevel
Area Host                           0
Board Member                        1
Full Membership                     4
Full Membership - Grandfathered     0
Full Membership - PayPal            2
Full Volunteer Authorizer           1
Grant Membership (Full)             0
Starving Hacker                    27
Starving Hacker - PayPal            7
Volunteer Authorizer                0
Name: Email, dtype: int64

# How many members are in good/bad standing?

In [10]:
# Data cleanup
pdf = pdf.sort_values(by="PaymentDate", ascending=False)
pdf['PaymentDate'] = pd.to_datetime(pdf['PaymentDate'])

# Calculate days sincel ast payment
latest_payment = pdf.groupby('ContactId').first().reset_index()
latest_payment['days_since_last'] = (pd.Timestamp.today() - latest_payment["PaymentDate"]).dt.days

# Merge with contact data
latest = latest_payment.merge(cdf, on="ContactId")

In [11]:
latest.head()

,ContactId,AllocatedValue,Comment,PaymentDate,PaymentId,TenderId,TenderName,Type,Value,days_since_last,Authorizations,Email,FirstName,LastName,MembershipLevel,MembeshipEnabled,Status,TermsOfUseAccepted
0,43719926,70.0,,2018-11-16,45547773,1159527.0,Stripe Recurring Payment,InvoicePayment,70.0,18,[],*****@****.***,*****,*****,Board Member,True,Active,True
1,43724423,30.0,Payment billing details:\nAndrew Camardella \n...,2018-12-04,45931504,1159526.0,Stripe,InvoicePayment,30.0,0,"[Boss Laser, Epilog Laser, Universal Laser, Sh...",*****@****.***,*****,*****,Board Member,True,Active,True
2,43724432,70.0,,2018-11-05,45316086,1159527.0,Stripe Recurring Payment,InvoicePayment,70.0,29,"[Boss Laser, Epilog Laser, ShopBot]",*****@****.***,*****,*****,Board Member,True,Active,True
3,43727162,70.0,,2018-10-20,44991418,1159527.0,Stripe Recurring Payment,InvoicePayment,70.0,45,"[Epilog Laser, ShopBot]",*****@****.***,*****,*****,Board Member,False,Lapsed,True
4,44170114,70.0,,2018-11-28,45741351,1159527.0,Stripe Recurring Payment,InvoicePayment,70.0,6,[],*****@****.***,*****,*****,Board Member,True,Active,False


In [12]:
good_standing = latest[latest['days_since_last'] <= 30]
bad_standing = latest[latest['days_since_last'] > 30]

In [13]:
good_standing.groupby("MembershipLevel")['Email'].count()

MembershipLevel
Board Member                         6
Full Membership                     54
Full Membership - Grandfathered      1
Full Volunteer Authorizer            4
Grant Membership (Full)              1
Starving Hacker                    200
Name: Email, dtype: int64

In [14]:
bad_standing.groupby("MembershipLevel")['Email'].count()

MembershipLevel
Area Host                    5
Board Member                 1
Full Membership              9
Starving Hacker             53
Starving Hacker - PayPal     3
Volunteer Authorizer         1
Name: Email, dtype: int64

# How many members do we have paying what amounts?

In [15]:
# Number of people paying different amounts at each membership level
latest.groupby(['MembershipLevel', 'Value'])['ContactId'].count()

MembershipLevel                  Value
Area Host                        30.0       1
                                 40.0       2
                                 70.0       2
Board Member                     30.0       1
                                 70.0       6
Full Membership                  70.0      63
Full Membership - Grandfathered  50.0       1
Full Volunteer Authorizer        30.0       4
Grant Membership (Full)          30.0       1
Starving Hacker                  30.0       1
                                 40.0     252
Starving Hacker - PayPal         40.0       3
Volunteer Authorizer             70.0       1
Name: ContactId, dtype: int64

# How many people are authorized for each thing?

In [16]:
adf.replace(False, 0).replace(True, 1).sum()

3D printers                   5
Band Saw                      4
Boss Laser                   74
Bridgeport Mill               3
CNC Plasma                    3
Clausing Lathe                1
Epilog Laser                109
Mig Welders                   3
Mitre Saw                     3
Panel Saw                     4
Pneumatic Power Tools         4
Powder Coating Equipment      2
Sanders                       4
ShopBot                     117
Table Saw                     6
Tig Welders                   2
Tormach CNC Mill              1
Tube Bending Equipment        1
Universal Laser              69
Wood Lathe                    1
dtype: int64

# What is quorum?

In [17]:
# Need voting data

# Who is eligible to run for the board?

# How many people voted in the last vote?

In [18]:
# Need voting data